# Financial Wellness Coach

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/robin-mongodb/mongochain/blob/main/examples/verticals/banking/financial_wellness_coach.ipynb)

A personalized banking agent that analyzes spending habits and helps users achieve savings goals.

## Features Demonstrated
- **Custom Tools**: Query transaction data, analyze spending trends, create savings plans
- **MongoDB Integration**: Mock transaction data stored in MongoDB
- **Memory Persistence**: Agent remembers user's financial goals
- **Personalized Advice**: Context-aware financial recommendations

## Prerequisites
- MongoDB Atlas cluster
- Voyage AI API key
- OpenAI API key

## 1. Setup

In [ ]:
# Install mongochain from GitHub
%pip install -q git+https://github.com/robin-mongodb/mongochain.git

In [ ]:
from getpass import getpass

# API Keys (input hidden)
MONGO_URI = getpass("MongoDB URI: ")
VOYAGE_API_KEY = getpass("Voyage API Key: ")
OPENAI_API_KEY = getpass("OpenAI API Key: ")

## 2. Seed Mock Transaction Data

Create realistic transaction data for the past 3 months. This data will be stored in the agent's MongoDB database.

In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import random

# User ID for this demo
USER_ID = "alex.chen@email.com"

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client["financial_wellness_coach"]  # Same as agent name
transactions = db["user_transactions"]

# Clear existing data for this user
transactions.delete_many({"user_id": USER_ID})

# Transaction categories with typical merchants and amount ranges
CATEGORIES = {
    "groceries": {
        "merchants": ["Whole Foods", "Trader Joe's", "Safeway", "Costco", "Target"],
        "amount_range": (25, 150),
        "frequency": 8  # times per month
    },
    "dining": {
        "merchants": ["Chipotle", "Starbucks", "Local Bistro", "Pizza Palace", "Thai Garden", "Coffee Bean"],
        "amount_range": (8, 75),
        "frequency": 12
    },
    "entertainment": {
        "merchants": ["Netflix", "Spotify", "AMC Theaters", "Steam Games", "Kindle Books"],
        "amount_range": (10, 60),
        "frequency": 5
    },
    "utilities": {
        "merchants": ["Electric Company", "Water Utility", "Internet Provider", "Phone Bill"],
        "amount_range": (50, 200),
        "frequency": 4
    },
    "transport": {
        "merchants": ["Shell Gas", "Uber", "Lyft", "Metro Transit", "Parking"],
        "amount_range": (15, 80),
        "frequency": 10
    },
    "shopping": {
        "merchants": ["Amazon", "Best Buy", "Nike", "Nordstrom", "Home Depot"],
        "amount_range": (20, 200),
        "frequency": 4
    },
    "health": {
        "merchants": ["CVS Pharmacy", "Gym Membership", "Doctor Visit Copay"],
        "amount_range": (15, 100),
        "frequency": 3
    }
}

# Generate 3 months of transactions
mock_transactions = []
today = datetime.now()

for month_offset in range(3):
    # Calculate the month
    month_start = today.replace(day=1) - timedelta(days=30 * month_offset)
    
    # Add some variation - more dining in recent month to show a trend
    dining_multiplier = 1.0 + (0.15 * (2 - month_offset))  # 15% more dining each month
    
    for category, config in CATEGORIES.items():
        frequency = config["frequency"]
        
        # Apply dining trend
        if category == "dining":
            frequency = int(frequency * dining_multiplier)
        
        for _ in range(frequency):
            # Random day in the month
            day = random.randint(1, 28)
            try:
                tx_date = month_start.replace(day=day)
            except ValueError:
                tx_date = month_start.replace(day=28)
            
            # Random merchant and amount
            merchant = random.choice(config["merchants"])
            amount = round(random.uniform(*config["amount_range"]), 2)
            
            mock_transactions.append({
                "user_id": USER_ID,
                "date": tx_date,
                "amount": amount,
                "category": category,
                "merchant": merchant
            })

# Insert transactions
transactions.insert_many(mock_transactions)
print(f"✅ Inserted {len(mock_transactions)} transactions for {USER_ID}")

# Show sample
print("\nSample transactions:")
for tx in list(transactions.find({"user_id": USER_ID}).limit(5)):
    print(f"  {tx['date'].strftime('%Y-%m-%d')} | ${tx['amount']:>7.2f} | {tx['category']:<12} | {tx['merchant']}")

## 3. Create the Financial Wellness Coach Agent

In [ ]:
from mongochain import MongoAgent

agent = MongoAgent(
    name="financial_wellness_coach",
    persona="""You are a friendly and knowledgeable Financial Wellness Coach.
Your role is to help users understand their spending habits, identify savings opportunities, 
and create actionable plans to achieve their financial goals.

Guidelines:
- Be encouraging but honest about spending patterns
- Provide specific, actionable advice
- Use the user's actual transaction data to personalize recommendations
- Remember the user's goals and reference them in future conversations
- When discussing money, always be precise with numbers
- Celebrate progress and small wins""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    llm_provider="openai"
)

print(f"\n{agent}")

## 4. Register Financial Tools

Create tools that query the transaction data and perform financial calculations.

### Tool 1 - Get Spending Summary

In [ ]:
from datetime import datetime, timedelta
from pymongo import MongoClient

# Reconnect for tool functions
_client = MongoClient(MONGO_URI)
_db = _client["financial_wellness_coach"]
_transactions = _db["user_transactions"]

def get_spending_summary(user_id: str, period: str = "current_month") -> str:
    """Get spending breakdown by category for a time period."""
    today = datetime.now()
    
    if period == "current_month":
        start_date = today.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        end_date = today
        period_label = "this month"
    elif period == "last_month":
        first_of_month = today.replace(day=1)
        end_date = first_of_month - timedelta(days=1)
        start_date = end_date.replace(day=1)
        period_label = "last month"
    elif period == "last_3_months":
        start_date = today - timedelta(days=90)
        end_date = today
        period_label = "the last 3 months"
    else:
        start_date = today.replace(day=1)
        end_date = today
        period_label = "this month"
    
    pipeline = [
        {"$match": {
            "user_id": user_id,
            "date": {"$gte": start_date, "$lte": end_date}
        }},
        {"$group": {
            "_id": "$category",
            "total": {"$sum": "$amount"},
            "count": {"$sum": 1}
        }},
        {"$sort": {"total": -1}}
    ]
    
    results = list(_transactions.aggregate(pipeline))
    
    if not results:
        return f"No transactions found for {period_label}."
    
    total = sum(r["total"] for r in results)
    summary = f"Spending summary for {period_label}:\n"
    summary += f"Total: ${total:.2f}\n\n"
    summary += "By category:\n"
    
    for r in results:
        pct = (r["total"] / total) * 100
        summary += f"- {r['_id'].capitalize()}: ${r['total']:.2f} ({pct:.1f}%) - {r['count']} transactions\n"
    
    return summary

# Register the tool
agent.register_tool(
    name="get_spending_summary",
    func=get_spending_summary,
    description="Get a breakdown of spending by category for a time period. Use this to answer questions about how much was spent on specific categories or overall.",
    parameters={
        "type": "object",
        "properties": {
            "user_id": {
                "type": "string",
                "description": "The user's email/ID"
            },
            "period": {
                "type": "string",
                "enum": ["current_month", "last_month", "last_3_months"],
                "description": "Time period to analyze"
            }
        },
        "required": ["user_id"]
    }
)

### Tool 2 - Get Spending Trends

In [ ]:
def get_spending_trends(user_id: str) -> str:
    """Compare spending between current and previous month to identify trends."""
    today = datetime.now()
    
    # Current month
    current_start = today.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    current_end = today
    
    # Last month
    last_month_end = current_start - timedelta(days=1)
    last_month_start = last_month_end.replace(day=1)
    
    def get_category_totals(start, end):
        pipeline = [
            {"$match": {
                "user_id": user_id,
                "date": {"$gte": start, "$lte": end}
            }},
            {"$group": {
                "_id": "$category",
                "total": {"$sum": "$amount"}
            }}
        ]
        return {r["_id"]: r["total"] for r in _transactions.aggregate(pipeline)}
    
    current = get_category_totals(current_start, current_end)
    previous = get_category_totals(last_month_start, last_month_end)
    
    # Calculate days elapsed for fair comparison
    days_in_current = (today - current_start).days + 1
    days_in_previous = (last_month_end - last_month_start).days + 1
    
    trends = []
    all_categories = set(current.keys()) | set(previous.keys())
    
    for cat in all_categories:
        curr_total = current.get(cat, 0)
        prev_total = previous.get(cat, 0)
        
        # Normalize to daily rate for fair comparison
        curr_daily = curr_total / days_in_current if days_in_current > 0 else 0
        prev_daily = prev_total / days_in_previous if days_in_previous > 0 else 0
        
        if prev_daily > 0:
            pct_change = ((curr_daily - prev_daily) / prev_daily) * 100
            if abs(pct_change) >= 10:  # Only report significant changes
                direction = "up" if pct_change > 0 else "down"
                trends.append({
                    "category": cat,
                    "change": pct_change,
                    "direction": direction,
                    "current": curr_total,
                    "previous": prev_total
                })
    
    if not trends:
        return "No significant spending trends detected. Your spending patterns are consistent with last month."
    
    # Sort by absolute change
    trends.sort(key=lambda x: abs(x["change"]), reverse=True)
    
    result = "Spending trends (compared to last month):\n\n"
    for t in trends:
        emoji = "📈" if t["direction"] == "up" else "📉"
        result += f"{emoji} {t['category'].capitalize()}: {t['direction']} {abs(t['change']):.1f}%\n"
        result += f"   Current month: ${t['current']:.2f} | Last month: ${t['previous']:.2f}\n\n"
    
    return result

agent.register_tool(
    name="get_spending_trends",
    func=get_spending_trends,
    description="Compare spending between current and previous month to identify trends and changes in spending behavior.",
    parameters={
        "type": "object",
        "properties": {
            "user_id": {
                "type": "string",
                "description": "The user's email/ID"
            }
        },
        "required": ["user_id"]
    }
)

### Tool 3 - Calculating Savings Plan

In [ ]:
def calculate_savings_plan(user_id: str, goal_amount: float, target_date: str, goal_name: str = "goal") -> str:
    """Calculate a savings plan based on current spending and a target goal."""
    from datetime import datetime
    
    # Parse target date
    try:
        target = datetime.strptime(target_date, "%Y-%m-%d")
    except ValueError:
        try:
            target = datetime.strptime(target_date, "%B %Y")
        except ValueError:
            return f"Could not parse date: {target_date}. Please use YYYY-MM-DD or 'Month Year' format."
    
    today = datetime.now()
    months_until = max(1, (target.year - today.year) * 12 + (target.month - today.month))
    
    # Calculate monthly savings needed
    monthly_savings = goal_amount / months_until
    weekly_savings = monthly_savings / 4.33
    
    # Get current month's spending to find savings opportunities
    current_start = today.replace(day=1)
    pipeline = [
        {"$match": {
            "user_id": user_id,
            "date": {"$gte": current_start}
        }},
        {"$group": {
            "_id": "$category",
            "total": {"$sum": "$amount"}
        }},
        {"$sort": {"total": -1}}
    ]
    
    spending = list(_transactions.aggregate(pipeline))
    
    result = f"💰 Savings Plan for {goal_name}: ${goal_amount:,.2f}\n"
    result += f"📅 Target date: {target.strftime('%B %Y')}\n"
    result += f"⏱️ Time remaining: {months_until} months\n\n"
    result += f"Required savings:\n"
    result += f"  • ${monthly_savings:,.2f} per month\n"
    result += f"  • ${weekly_savings:,.2f} per week\n\n"
    
    # Suggest where to cut
    discretionary = ["dining", "entertainment", "shopping"]
    disc_spending = sum(s["total"] for s in spending if s["_id"] in discretionary)
    
    if disc_spending > 0:
        result += f"💡 Potential savings opportunities:\n"
        result += f"Your discretionary spending (dining, entertainment, shopping) this month: ${disc_spending:.2f}\n"
        
        if disc_spending > monthly_savings:
            cut_pct = (monthly_savings / disc_spending) * 100
            result += f"Reducing discretionary spending by {cut_pct:.0f}% would achieve your goal!\n"
        else:
            result += f"Consider reviewing subscriptions or finding one area to reduce.\n"
    
    return result

agent.register_tool(
    name="calculate_savings_plan",
    func=calculate_savings_plan,
    description="Create a savings plan for a financial goal. Calculates monthly/weekly savings needed and suggests where to cut spending.",
    parameters={
        "type": "object",
        "properties": {
            "user_id": {
                "type": "string",
                "description": "The user's email/ID"
            },
            "goal_amount": {
                "type": "number",
                "description": "Target savings amount in dollars"
            },
            "target_date": {
                "type": "string",
                "description": "Target date in YYYY-MM-DD or 'Month Year' format"
            },
            "goal_name": {
                "type": "string",
                "description": "Name of the savings goal (e.g., 'vacation', 'emergency fund')"
            }
        },
        "required": ["user_id", "goal_amount", "target_date"]
    }
)

### Tool 4 - Get Recent Transactions

In [ ]:
def get_recent_transactions(user_id: str, count: int = 10, category: str = None) -> str:
    """Get recent transactions, optionally filtered by category."""
    query = {"user_id": user_id}
    if category:
        query["category"] = category.lower()
    
    txs = list(_transactions.find(query).sort("date", -1).limit(count))
    
    if not txs:
        if category:
            return f"No transactions found in category: {category}"
        return "No transactions found."
    
    result = f"Recent transactions"
    if category:
        result += f" ({category})"
    result += f":\n\n"
    
    total = 0
    for tx in txs:
        result += f"{tx['date'].strftime('%Y-%m-%d')} | ${tx['amount']:>7.2f} | {tx['category']:<12} | {tx['merchant']}\n"
        total += tx["amount"]
    
    result += f"\nTotal: ${total:.2f}"
    return result

agent.register_tool(
    name="get_recent_transactions",
    func=get_recent_transactions,
    description="Get a list of recent transactions, optionally filtered by category.",
    parameters={
        "type": "object",
        "properties": {
            "user_id": {
                "type": "string",
                "description": "The user's email/ID"
            },
            "count": {
                "type": "integer",
                "description": "Number of transactions to return (default: 10)"
            },
            "category": {
                "type": "string",
                "description": "Filter by category (e.g., 'dining', 'groceries')"
            }
        },
        "required": ["user_id"]
    }
)

print("✅ All financial tools registered!")

## 5. Demo: Understanding Spending Habits

Ask the agent about spending patterns. The agent will use the registered tools to query real transaction data.

In [ ]:
# Ask about spending summary
response = agent.chat(
    user_id=USER_ID,
    message="Hi! Can you give me a breakdown of my spending this month? I want to understand where my money is going."
)

print(response)

In [ ]:
# Ask about spending trends
response = agent.chat(
    user_id=USER_ID,
    message="Have my spending habits changed recently? Any trends I should be aware of?"
)

print(response)

## 6. Demo: Setting a Savings Goal

Ask the agent to help plan for a specific financial goal. The agent will calculate a savings plan and remember this goal.

In [ ]:
# Set a savings goal
response = agent.chat(
    user_id=USER_ID,
    message="I want to save $2,000 for a vacation to Hawaii by June 2025. Can you help me create a plan?"
)

print(response)

## 7. Demo: Memory-Aware Follow-up

The agent remembers the user's savings goal. Ask a follow-up question without mentioning the goal explicitly.

In [ ]:
# Follow-up question - agent should remember the Hawaii goal
response = agent.chat(
    user_id=USER_ID,
    message="If I cut back on dining out, would that help me reach my goal faster?"
)

print(response)

In [ ]:
# Ask the agent to summarize what it knows
response = agent.chat(
    user_id=USER_ID,
    message="Can you remind me of my financial goals and what we've discussed?"
)

print(response)

## 8. Ending User Session

Ending the user session to force a short term memory creation

In [ ]:
# Ending the user session to force a short term memory creation
agent.end_session(USER_ID)

## 9. View MongoDB Collections

Explore what's stored in MongoDB — transactions, agent metadata, tools, and user memories.

In [ ]:
# View agent metadata
print("=== Agent Metadata ===")
metadata = agent.get_metadata()
if metadata:
    print(f"Name: {metadata.get('name')}")
    print(f"LLM: {metadata.get('llm_provider')} / {metadata.get('llm_model')}")
    print(f"Tools: {metadata.get('tools', [])}")
    print(f"Created: {metadata.get('created_at')}")

# View registered tools
print("\n=== Registered Tools ===")
tools = agent.get_tools()
for tool in tools:
    print(f"  • {tool['name']}: {tool['description'][:60]}...")

# View user stats
print(f"\n=== User Memory Stats ({USER_ID}) ===")
stats = agent.get_user_stats(USER_ID)
print(f"  Conversations: {stats['conversation_count']}")
print(f"  Short-term memories: {stats['short_term_count']}")
print(f"  Long-term memories: {stats['long_term_count']}")

# View extracted memories
print(f"\n=== Extracted User Memories ===")
memories = agent.get_user_memories(USER_ID, limit=5)
for m in memories:
    print(f"  [{m.get('category', 'general')}] {m['content']}")